In [10]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import joblib

# Cargar el dataset
data = pd.read_excel('/content/drive/MyDrive/2 -2024/Sistemasa-Embebidos-2/Laboratorio_10/Jugadores.xlsx')

# Inspeccionar los datos
print(data.head())

# Limpiar los datos (por ejemplo, eliminar filas con valores faltantes)
data = data.dropna()

# Seleccionar las características y la etiqueta
features = ['Posicion Num', 'Edad', 'Partidos Jugados', 'Goles', 'Asistencias', 'Penales Acertados',
            'Tarjetas Amarillas', 'Tarjetas Rojas', 'Goles Esperados']
X = data[features]
y = data['Label']

# Normalizar los datos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Guardar el scaler para usarlo después
joblib.dump(scaler, 'scaler.save')


   Numero de Jugador            Jugador Pais Posicion  Posicion Num  Edad  \
0           0.797908  Cristiano Ronaldo  POR     DLCC            43    29   
1           0.614087              Djené  TOG       DF             2    29   
2           0.514732            Rodrigo  ESP       DL             4    21   
3           0.866826         Allan Nyom  CMR       DF             2    33   
4           0.839183   Javier Ontiveros  ESP     CCDL            34    23   

   Partidos Jugados  Titular  Minutos  Goles  ...  Goles Sin Penalizacion  \
0                35       35     3100     48  ...                      38   
1                35       35     3115      0  ...                       0   
2                21       10      881      1  ...                       1   
3                 5        3      244      0  ...                       0   
4                 3        0       39      0  ...                       0   

   Penales Acertados  Total Penales  Tarjetas Amarillas  Tarjetas Rojas  \

['scaler.save']

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Crear el modelo
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_scaled.shape[1],)),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Salida binaria
])

# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Definir early stopping para evitar sobreajuste
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Entrenar el modelo
history = model.fit(X_scaled, y, epochs=50, validation_split=0.2, batch_size=32, callbacks=[early_stopping])

# Guardar el modelo
model.save('modelo_jugadores.h5')


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.4720 - loss: 0.7694 - val_accuracy: 0.8435 - val_loss: 0.5938
Epoch 2/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8002 - loss: 0.5562 - val_accuracy: 0.8783 - val_loss: 0.4658
Epoch 3/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8594 - loss: 0.4597 - val_accuracy: 0.8783 - val_loss: 0.4035
Epoch 4/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8775 - loss: 0.4283 - val_accuracy: 0.8783 - val_loss: 0.3764
Epoch 5/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8804 - loss: 0.3980 - val_accuracy: 0.8783 - val_loss: 0.3676
Epoch 6/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8992 - loss: 0.3883 - val_accuracy: 0.8783 - val_loss: 0.3654
Epoch 7/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8958 - loss: 0.3658 - val_accuracy: 0.8783 - val_loss: 0.3666
Epoch 8/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8975 - loss: 0.3333 - val_accuracy: 0.8783 - val_loss: 0.3681
Ep

In [12]:
import numpy as np
from tensorflow.keras.models import load_model

# Cargar el modelo entrenado y el scaler ajustado
model = load_model('modelo_jugadores.h5')
scaler = joblib.load('scaler.save')

# Definir los datos de los jugadores
jugadores = np.array([
    [2, 28, 30, 1, 0, 0, 9, 1, 1.7],  # Harry Maguire
    [4, 33, 32, 27, 12, 7, 0, 0, 23.5],  # Karim Benzema
    [4, 32, 34, 35, 3, 5, 2, 0, 32],  # Robert Lewandowski
    [2, 31, 18, 0, 0, 0, 4, 1, 0.5]   # Ximo Navarro
])

# Normalizar los datos de los jugadores con el scaler ajustado
jugadores_scaled = scaler.transform(jugadores)

# Hacer las predicciones
predicciones = model.predict(jugadores_scaled)

# Mostrar los resultados
for i, pred in enumerate(predicciones):
    print(f'Jugador {i+1}: {"Posible adquisición" if pred >= 0.5 else "No recomendado"} (Probabilidad: {pred[0]:.2f})')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
Jugador 1: No recomendado (Probabilidad: 0.03)
Jugador 2: No recomendado (Probabilidad: 0.02)
Jugador 3: No recomendado (Probabilidad: 0.01)
Jugador 4: No recomendado (Probabilidad: 0.03)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
